In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
 
!git clone https://$BITBUCKET_AUTH@bitbucket.org/reisert/patchwork.git
!mkdir models

import sys 
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as npss
import tensorflow as tf
import math
import json
from tensorflow.keras import Model
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import patchwork as patchwork



# Download data 

In [ ]:
# Download data from medical decathlon repo
taskname = 'Task04_Hippocampus'
!gdown --id 1RzPB1_bqzQhlWvU-YGvZzhx2omcDh38C
!tar -xf Task04_Hippocampus.tar

# prepare json for patchwork
dataset = json.load(open(taskname + '/dataset.json'))
contrasts = {}
labels = {}
items = []
label_indices = list(map(int,dataset['labels'].keys()))[1:]
cnt = 1
for obj in dataset['training']:
     id = 'item%i' % cnt
     contrasts[id] = taskname + obj['image'][1:]
     labels[id] =  taskname + obj['label'][1:]
     items.append(id)
     cnt = cnt + 1


# Define the model

In [ ]:

modelfi = "models/" + taskname  # here our model is saved
#reinit_model = True

# dim of problem (2D/3D)
nD = 3

### PATCHING OPTIONS
patching = {        
    "depth":3,                    
    "scheme":{ 
        "patch_size":[32,32,32],                
        "destvox_mm": None,
        "destvox_rel":[1,1,1],
        "fov_mm":None,
        "fov_rel":[0.8,0.8,0.8],
     },
    "categorial_label" :label_indices,
    "normalize_input" : 'patch_m0s1',
    }

### NETWORK OPTIONS
network = {    
    "blockCreator": lambda level,outK,input_shape : 
        patchwork.customLayers.createUnet_v2(depth=3,outK=outK,nD=nD,
                                             input_shape=input_shape,
                                             feature_dim=[4,8,8,8,16],
                                             nonlin='relu',
                                             ),
    "finalBlock": tf.keras.layers.Activation('sigmoid'),
    "intermediate_loss":True,          
    "forward_type":"bridge",
    }

## DATA IMPORT OPTIONS
loading = {
    "crop_fdim":None,
    "crop_fdim_labels":None,
    "integer_labels":True,      # needed for categorial/index label images
    "threshold":None,
    }



def get_data(n=None):
   return patchwork.improc_utils.load_data_structured(
                       contrasts=[contrasts],labels=[labels],subjects=items,max_num_data=n, **loading)

print("\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loading first example for init.")

with tf.device("/cpu:0"):    
    tset,lset,rset,subjs = get_data(1)
    
if len(tset) == 0:
    print("Error: No data found!")
    raise NameError('No data found! Stopping ...  ')

# load or generate model
if os.path.isfile(modelfi+".json") and not reinit_model:
    print("\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> model already existing, loading ")
    themodel = patchwork.PatchWorkModel.load(modelfi,immediate_init=True,notmpfile=True)
else:

    print("\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> creating new model")    
    
    patching['ndim'] = nD    
    network['modelname'] = modelfi  

    if 'finalBlock' not in network:
        network['finalBlock']=layers.Activation('sigmoid')
    if patching['categorial_label'] is not None:
        network['num_labels']= len(patching['categorial_label'])
    else:
        network['num_labels']= lset[0].shape[nD+1]
    print('numlabels:' + str(network['num_labels']))
    
    # create model
    cgen = patchwork.CropGenerator(**patching)
    themodel = patchwork.PatchWorkModel(cgen, **network)
    
    print("\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>> initializing network")
    dataexample = tset[0][0:1,...]    
    tmp = themodel.apply_full(dataexample,resolution=rset[0],repetitions=100,generate_type='random',verbose=True)
    
    

print("\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>> model summary")
themodel.summary()


# Training of the model

In [ ]:



## TRAINING OPTION
training = {
   "num_patches":25,
   "augment": {"dphi":0.2, "flip":[0,0,0] , "dscale":[0.1,0.1,0.1] },
   "epochs":5,
   "num_its":100,                
   "balance":{"ratio":0.5,"autoweight":1},

   "debug":False,
   "patch_on_cpu":True,
   #"hard_mining":0.2,
   #"hard_mining_order":'balance',

   }


# define, if you want, some validation data
valid_ids = []

outer_num_its = 10
num_samp = 10
print("\n\n\n\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> starting training")
import gc
for i in range(0,outer_num_its):
    
    print("\n========================================================= loading data =========================================================")
    with tf.device("/cpu:0"):    
        tset,lset,rset,subjs = get_data(num_samp)

    themodel.train(tset,lset,resolutions=rset,**training,
                   verbose=2,
                   valid_ids=valid_ids)
    
    with tf.device("/cpu:0"):  
        del tset
        del lset
        gc.collect()
    sys.stdout.flush()


# Apply the model

In [ ]:
#%%
finame = taskname + dataset['test'][0][1:]
ew =    themodel.apply_on_nifti(finame,'exmample.nii',repetitions=250,num_chunks=1,
                                generate_type='random',
                                augment={"dphi":0.2,'independent_augmentation':False},
                                branch_factor=2,lazyEval=0.5,                                
                                scale_to_original=False)


plt.imshow(tf.squeeze(ew[1][:,:,10,0]))
